2.Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты, то есть цифра вводится одна, а запрос проверяет оба поля)

In [1]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd

In [2]:
#Python
world = input('Введите значение слова поиска: ')

Введите значение слова поиска: Python


In [24]:
smin = int(input('Введите значение зарплаты, менше которой вакансии нас не интересуют(в рублях): '))

Введите значение зарплаты, менше которой вакансии нас не интересуют(в рублях): 100000


In [6]:
Name = ['Вакансия','Ссылка на вакансию','Минимальна зароботная плата','Фиксированная ставка','Максимальная зароботная плата','Валюта'] 
   

In [7]:
def parser_hh(world,smin):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    params = {'page': 1}
    url = "https://hh.ru"
    #Name = ['Вакансия','Ссылка на вакансию','Минимальна зароботная плата','Фиксированая ставка','Максимальная зароботная плата','Валюта'] 
    vacancy_date = []
    session = requests.Session()
    n=1
    response = session.get('https://hh.ru/search/vacancy?text='+world+'&from=suggest_post', headers=headers, params=params)
   # У меня очень тормозит комп, поэтому можно заменить строку 'while response and n<3:' на строку 'while response:'
    while response and n<5:
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
        for article in articles:
            info = article.find('a', {'class': 'serp-item__title'})
            vacname=info.text
            link = article.find('a').get('href')
            salary = article.find('span', {'class': 'bloko-header-section-2'}) 
            is_check_sarary =''
            salary_currency = None
            salary_max = None
            salary_min = None
            vtmp1 =''
            vtmp2 =''
            v= ''
            if not salary:
                pass
            else:
                is_check_sarary = salary.text.replace(u'\u202f', u' ')
                v = is_check_sarary[is_check_sarary.rfind(' ')+1:]
                is_check_sarary =is_check_sarary.replace(v, '')
                is_check_sarary =is_check_sarary.replace(' ', '')
                k =0
                for i in is_check_sarary:
                    if i.isdigit() and k==0:
                        vtmp1=vtmp1+i
                    elif i == ' ' and k==0:
                        pass
                    elif vtmp1 !='':
                        k=1
                    else:
                        pass
                k=0
                for i in reversed(is_check_sarary): 
                    if i.isdigit() and k==0:
                        vtmp2=i+vtmp2
                    elif i == ' ' and k==0:
                        pass
                    elif vtmp2 !='':
                        k=1
                    else:
                        pass
                if v=='₽':
                    if 'до' in is_check_sarary:
                        salary_max = int(vtmp2)
                        if smin < int(vtmp2):
                            vacancy_date.append([vacname,link,salary_min,salary_currency,salary_max,v])
                    elif 'от' in is_check_sarary:
                        salary_min = int(vtmp1)
                        if smin<int(vtmp1):
                            vacancy_date.append([vacname,link,salary_min,salary_currency,salary_max,v])
                        #salary_max = ''
                    elif '–' in is_check_sarary:
                        if smin<int(vtmp1) and smin<int(vtmp2):
                            salary_min = int(vtmp1)
                            salary_max = int(vtmp2)
                            vacancy_date.append([vacname,link,salary_min,salary_currency,salary_max,v])
                    else:
                        if smin<int(vtmp1):
                            salary_currency = int(vtmp1)
                            vacancy_date.append([vacname,link,salary_min,salary_currency,salary_max,v])
                    
            n=n+1
            params['page']+=1
            response = session.get('https://hh.ru/search/vacancy?text='+world+'&from=suggest_post', headers=headers, params=params)
        return vacancy_date
    #print(n)
    #dframe = pd.DataFrame(vacancy_date, columns=Name)
    #dframe

In [8]:
print(f"Собрана информация о вакансиях с сата https://hh.ru по ключевому слову {world}.")

Собрана информация о вакансиях с сата https://hh.ru по ключевому слову Python.


In [9]:
dframe = pd.DataFrame(parser_hh(world,smin), columns=Name)
dframe

,Вакансия,Ссылка на вакансию,Минимальна зароботная плата,Фиксированная ставка,Максимальная зароботная плата,Валюта
0,Преподаватель программирования в колледж,https://adsrv.hh.ru/click?b=472541&c=6&place=3...,65000,None,120000.0,₽
1,Middle/Senior Python developer,https://hh.ru/vacancy/83808487?from=vacancy_se...,200000,None,350000.0,₽
2,Middle разработчик Python / Django,https://hh.ru/vacancy/82910400?from=vacancy_se...,60000,None,140000.0,₽
3,Middle Backend разработчик (Python + Golang),https://hh.ru/vacancy/82915680?from=vacancy_se...,200000,None,285000.0,₽
4,"Senior Python Developer (AI, robotics)",https://hh.ru/vacancy/81940901?from=vacancy_se...,250000,None,500000.0,₽
5,Программист Python,https://hh.ru/vacancy/82795669?from=vacancy_se...,60000,None,NaN,₽
6,"Разработчик Python (FastAPI, MySQL, Clickhouse...",https://hh.ru/vacancy/82826027?from=vacancy_se...,180000,None,270000.0,₽
7,Программист Python,https://hh.ru/vacancy/83028791?from=vacancy_se...,60000,None,120000.0,₽
